# Using Dotnet Interactive Feature to run .NET code in Jupyter Notebook

<img src="https://ms-dotnettools.gallerycdn.vsassets.io/extensions/ms-dotnettools/dotnet-interactive-vscode/1.0.5565011/1731690631035/Microsoft.VisualStudio.Services.Icons.Default" alt="image" width="50"/>

Polyglot Notebook is a feature of the .NET Interactive ecosystem. It allows you to work interactively with multiple programming languages (like Python, C#, F#, JavaScript, SQL, etc.) within the same notebook environment. This is particularly useful for scenarios that require multi-language workflows, such as data analysis, machine learning, and integrating diverse systems.

For more Context [PolyGLot](https://github.com/dotnet/interactive/blob/main/docs/NotebookswithJupyter.md)

Make sure `.NET Interactive` choosen as KERNEL

First Download The Required Nuget Package

In [ ]:
#r "nuget: CsvHelper,33.0.1"
#r "nuget: MoreLinq,4.3.0"
#r "nuget: TensorFlow.NET, 0.150.0"
#r "nuget: TensorFlow.Keras, 0.15.0"

The second part is the computing support part. Only one of the following packages is needed, depending on your device and system.

In [ ]:

//CPU version for Windows and Linux
#r "nuget: SciSharp.TensorFlow.Redist,2.16.0"

// CPU version for MacOS
//#r "nuget: SciSharp.TensorFlow.Redist-OSX, 2.15.0"

// GPU version for Windows (CUDA and cuDNN are required)
//#r "nuget: SciSharp.TensorFlow.Redist-Windows-GPU, 2.10.3"

// GPU version for Linux (CUDA and cuDNN are required)
//#r "nuget: SciSharp.TensorFlow.Redist-Linux-GPU, 2.11.1"

In [ ]:
// Global Parameter 
const int imgH = 32 ; 
const int imgW = 32 ;
const int nChannels = 3 ;

In [ ]:
using System;
using System.IO;

In [ ]:
Console.WriteLine(Directory.GetCurrentDirectory());

In [ ]:
string basePath = Path.Combine(Directory.GetCurrentDirectory(), "Data");


In [ ]:
using CsvHelper;
using MoreLinq.Extensions;
using System.Globalization;
using Tensorflow;
using Tensorflow.IO;
using Tensorflow.Keras.Engine;
using Tensorflow.Keras.Layers;
using Tensorflow.NumPy;
using static Tensorflow.Binding;

In [ ]:
var result = Directory.GetDirectories(basePath);
foreach (var x in result)
{
    Console.WriteLine(x);
}

In [ ]:
// Model Class to read & Deserialize CSV file 
public class DataRow
{
    //public int Width { get; set; }
    //public int Height { get; set; }
    //public int RoiX1 { get; set; }
    //public int RoiY1 { get; set; }
    //public int RoiX2 { get; set; }
    //public int RoiY2 { get; set; }
    public int ClassId { get; set; }
    public string Path { get; set; }
}

public sealed class DataRowMap : CsvHelper.Configuration.ClassMap<DataRow>
{
    // Only Required ClassId & Path
    public DataRowMap()
    {
        //Map(m => m.Width).Name("Width");
        //Map(m => m.Height).Name("Height");
        //Map(m => m.RoiX1).Name("Roi.X1");
        //Map(m => m.RoiY1).Name("Roi.Y1");
        //Map(m => m.RoiX2).Name("Roi.X2");
        //Map(m => m.RoiY2).Name("Roi.Y2");
        Map(m => m.ClassId).Name("ClassId");
        Map(m => m.Path).Name("Path");
    }
}




In [ ]:

/// <summary>
/// Read lists of images and return as NDArray of size (batchSize,height,width,depth)
/// </summary>
/// <param name="a">Array of Image Path</param>
/// <param name="b">An Empty NDArray</param>
/// <param name="process">Name of process</param>
void LoadImage(string[] a, NDArray b, string process)
    {
        // Reading Images ConCurrently using Task Parallel Library(TPL)
        Parallel.For(0, a.Length, (i) =>
        {
            try
            {
                var graph = tf.Graph().as_default();
                b[i] = ReadTensorFromImageFile(a[i], graph);
                Console.WriteLine($"Loading image: {i} {a[i]}...");
                Console.CursorLeft = 0;
                graph.Exit();
            }
            catch(Exception ex) { Console.WriteLine(ex.Message);}
        });

        Console.WriteLine();
        Console.WriteLine($"Loaded {a.Length} images for " + process);
    }



private NDArray ReadTensorFromImageFile(string fileName, Graph graph)
    {
        var fileReader = tf.io.read_file(fileName, "file_reader");
        var decodeImage = tf.image.decode_jpeg(fileReader, channels: 3, name: "DecodeJpeg");
        //var decodeImage = tf.image.decode_image(fileReader, channels: 3, name: "DecodeImage");
        // Change Format to Float32 bit
        var cast = tf.cast(decodeImage, tf.float32,"cast");
        // Not Required
        //resize required one extra dims
        var dims_expander = tf.expand_dims(cast, 0);

        var resize = tf.constant(new int[] { 32, 32 }, name: "resize");

        var bilinear = tf.image.resize_bilinear(dims_expander, resize);//(dims_expander, resize);
        var sub = tf.subtract(bilinear, new float[] { 0 });
        var normalized = tf.divide(sub, new float[] { 255 });

        var sess = tf.Session(graph);
        return sess.run(normalized);

    }





List<DataRow> ReadCsv(string path)
    {
       
        using var reader = new StreamReader(path);
        using var csv = new CsvReader(reader, CultureInfo.InvariantCulture);
        //csv.Context.RegisterClassMap<DataRowMap>();
        return [..csv.GetRecords<DataRow>().Shuffle()]; ;
    }

void CreateImage(Dictionary<string, List<float>> history, string path)
    {

        foreach (var (name,data) in history)
        {

        }
    }

In [ ]:

public class TrafficSignal
{
    private ILayersApi layers = tf.keras.layers;

    private IModel model { get; set; }

    /// <summary> Build CNN Model</summary>
    public void BuildModel(int height, int width, int depth, int classNumber)
    {
        // Check the Diff in GlobalAveragePooling2D() & AveragePooling2D()
        // input layer
        //keras.Sequential()
        var inputs = layers.Input(shape: (height, width, depth), name: "img");  //(32, 32, 3), name: "img");

        // convolutional layer
        var x = layers.Conv2D(8, (5, 5), padding: "same", activation: "relu").Apply(inputs);
        x = layers.BatchNormalization().Apply(x);
        x = layers.MaxPooling2D(pool_size: (2, 2)).Apply(x);

        x = layers.Conv2D(16, kernel_size: (3, 3), activation: "relu", padding: "same").Apply(x);
        x = layers.BatchNormalization().Apply(x);

        x = layers.Conv2D(16, kernel_size: (3, 3), activation: "relu", padding: "same").Apply(x);
        x = layers.BatchNormalization().Apply(x);
        x = layers.MaxPooling2D(pool_size: (2, 2)).Apply(x);

        x = layers.Conv2D(32, kernel_size: (3, 3), activation: "relu", padding: "same").Apply(x);
        x = layers.BatchNormalization().Apply(x);

        x = layers.Conv2D(32, kernel_size: (3, 3), activation: "relu", padding: "same").Apply(x);
        x = layers.BatchNormalization().Apply(x);
        x = layers.MaxPooling2D(pool_size: (2, 2)).Apply(x);

        x = layers.Flatten().Apply(x);
        x = layers.Dense(128, activation: "relu").Apply(x);
        x = layers.BatchNormalization().Apply(x);
        x = layers.Dropout(0.5f).Apply(x);

        x = layers.Flatten().Apply(x);
        x = layers.Dense(128, activation: "relu").Apply(x);
        x = layers.BatchNormalization().Apply(x);
        x = layers.Dropout(0.5f).Apply(x);

        // output layer
        var outputs = layers.Dense(classNumber, "softmax").Apply(x);
        // build keras model
        model = tf.keras.Model(inputs, outputs, name: "traffic_resnet");
    }

    /// <summary> 
    /// Train Build CNN model.
    /// Make sure to run `BuildModel` method before running this method.
    /// </summary>
    public ICallback Train(NDArray xTrain, NDArray yTrain,Dictionary<int,float> classWeight=null)
    {
        // training
        return model!.fit(xTrain, yTrain,
        batch_size: 64,
            epochs: 10,
            validation_split: 0.2f,
            class_weight: classWeight);
    }

    /// <summary>
    /// Generate Detail of Trained model
    /// </summary>
    public void Summary()
    {
        if (model is null)
            throw new NullReferenceException("First call `BuildModel` Method to INITIALIZED the model object");
        model.summary();
    }

    /// <summary>
    /// Compile the model
    /// </summary>
    public void Compile()
    {
        if (model is null)
            throw new NullReferenceException("First call `BuildModel` Method to INITIALIZED the model object");
        
        model!.compile(optimizer: tf.keras.optimizers.RMSprop(1e-3f),
            loss: tf.keras.losses.CategoricalCrossentropy(from_logits:false), // SparseCategoricalCrossentropy(from_logits: true),
            metrics: [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.CategoricalCrossentropy()]); //new[] { "acc" }); // //
    }
    //metrics_utils

    /// <summary>
    /// Save trained Model weight
    /// </summary>
    public void Save(string filePath) //"./toy_resnet_model"
    {
        if (model is null)
            throw new NullReferenceException("First call `BuildModel` Method to INITIALIZED the model object");
                
        // save the model
        model!.save(filePath,save_format:"tf");
        
    }

    /// <summary>
    /// Run prediction based on Trained model
    /// </summary>
    public Tensor Predict(Tensor value,int verbose =0)
    {
        // var c = confusion_matrix;
        var result = model.predict(value, verbose: verbose);
        return tf.arg_max(result, 1);
    }

    public void LoadMode(string modelPath){
        if(String.IsNullOrEmpty(modelPath))
            throw new NullReferenceException("Please Provide the Path");
            
       model = tf.keras.models.load_model(modelPath);
       Console.WriteLine("Loding Model...");
       model.summary();
       Compile();
    }
}



In [ ]:
List<DataRow> records = ReadCsv(path: Path.Join(basePath, "Train.csv"));
Console.Write(records.Count())

In [ ]:


// 39209
List<int> yLabels = [];
List<string> xImagePath = [];

foreach (var (index, row) in records.Select((row, index) => (index, row)))
{
    if (index % 10000 == 0)
    {
        Console.WriteLine($"[INFO] processed {index} total images");
    }

    //var (label, imagePath) = (row.ClassId, row.Path);

    yLabels.add(row.ClassId);
    xImagePath.add(Path.Combine(basePath, row.Path));

    //imagePath = Path.Combine(BasePath, imagePath);

    //NDArray a = np.array<int>(xLabels.ToArray());
}


In [ ]:

int[] uniqueLabels = yLabels.Distinct().ToArray();
int classCount = uniqueLabels.Count();
int totalCount = yLabels.Count();


In [ ]:
Dictionary<int,float> classWeight = yLabels.GroupBy(x => x)
    .Select(g => new { Index = g.Key, Count = g.Count() })
    .OrderBy(x=>x.Index)
    .ToDictionary(x => x.Index, x =>  totalCount/ (float)(classCount*x.Count));
//.ToDictionary(x => x.Index, x => x.Count/ (float)totalCount );


In [ ]:

Console.WriteLine("ClassWeight");
float total = 0f;
foreach (var (key, value) in classWeight)
{
    Console.WriteLine($"|     {key,-8}|  {value}");
    total += value;
}
Console.WriteLine("".PadRight(20,'='));
Console.WriteLine($"|    TOTAL    | {total}");


In [ ]:

//Create Empty
// TF message comming from here
var xTrain = np.zeros((records.Count, imgH, imgW, nChannels), dtype: tf.float32); // TotalRecords * Height * width * Channel
//var yTrain = tf.one_hot(np.array(xLabels.ToArray(),dtype:tf.int64), depth: classCount);
var yTrain = np.eye(classCount, dtype: tf.float32)[np.array(yLabels.ToArray(), tf.float32).reshape(-1)];
// Encode label to a one hot vector.

//var indexArray = np.array(xLabels.ToArray());  // N * xLabels.Total

//var one = yTrain[indexArray];

//indexArray = indexArray.reshape(-1);

//var one_hot_targets = np.eye(uniqueLabels.Length)[indexArray];
//var sh = one_hot_targets.shape;
//Load labels


In [ ]:

//Util.ToCategorical(y_train, num_classes);
print("Load Labels To NDArray : OK!");
int i = 0;
// TO Check the Value 
foreach (var val in yTrain[0])
{
    Console.Write($"{val} ");
    i++;
}

Console.WriteLine(yTrain[0].shape);



In [ ]:
LoadImage(xImagePath.ToArray(), xTrain, "Training");

In [ ]:
tf.image.decode_image(xtrain[0],3,tf.float32,"Train1.png");

In [ ]:
TrafficSignal ts = new TrafficSignal();

In [ ]:


ts.BuildModel(imgH,imgW, nChannels, classCount);

ts.Compile();



In [ ]:

var startTime = DateTime.Now;
var history = ts.Train(xTrain, yTrain, classWeight);
var endTime = DateTime.Now;
var diff = endTime - startTime;
Console.WriteLine($"Execution Time {diff.Minutes} {diff.Seconds} {diff.Milliseconds}");

In [ ]:

ts.Summary();


In [ ]:
ts.Save("./Model");

In [ ]:
ts.LoadMode(Path.Combine(Directory.GetCurrentDirectory(), "Model"));

In [ ]:
// var hist = JsonConvert.SerializeObject(history.history);
// File.WriteAllText("History.json",hist);
// r.CreateImage(history.history,path:Path.Join(BasePath,"..","ModelResult.jpg"));



In [ ]:

/* NOW TEST THE MODEL*/

//FilePath = Path.Combine(BasePath, "Test");

records = r.ReadCsv(path: Path.Combine(BasePath ,"Test.csv"));

List<string> testImagePath =new();
List<int> textXLabels = new();

foreach (var row in records)
{
    //yTest.add(row.ClassId);
    testImagePath.add(Path.Combine(basePath, row.Path));
    textXLabels.add(row.ClassId);
}


// compile keras model in tensorflow static graph

// prepare dataset
// normalize the input
// x_train = x_train / 255.0f;
var xTest = np.zeros((testImagePath.Count, imgH, imgW, nChannels), dtype: tf.float32);

LoadImage(testImagePath.ToArray(), xTest, "Testing");

var yTest=ts.Predict(xTest,1);

Console.WriteLine(yTest[0]);


Console.WriteLine(yTest);
var yNDarray = yTest.numpy();
Console.WriteLine($"{yNDarray[0]}");



In [ ]:

// Create Confusion Matrix

List<List<int>> matrix = new();
for (int ind = 0; ind < classCount; ind++)
{
    matrix.add(Enumerable.Repeat(0,classCount).ToList());
}
Console.WriteLine();

for (i = 0 ;i< textXLabels.Count; i++)
{
    try
    {
        int j = yNDarray[i];              //yTest.numpy()[i];
        matrix[textXLabels[i]][j] += 1;
    }
    catch (Exception ex)
    {
        Console.WriteLine();
    }
}
Console.WriteLine();


//var m = JsonConvert.SerializeObject<List<List<string>>>(matrix);
//File.WriteAllText("Metrix.json", m);

Console.WriteLine("Predict Value - >");
foreach (var row in matrix)
{
    foreach (var col in row)
    {
        Console.Write($"{col} ");
    }
}


In [ ]:
#r "nuget: Plotly.NET.Interactive, 5.0.0"

using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Plotly.NET;
using Plotly.NET.LayoutObjects;

In [ ]:
#r "nuget: Newtonsoft.Json"
using Newtonsoft.Json;
using System.IO;
var matrix = File.ReadAllText("Metrix.json");

var m = JsonConvert.DeserializeObject<List<List<int>>>(matrix);

In [ ]:
// foreach(var r in m){
//     foreach(var x in r)
//     {
//         Console.Write("["+$"{x}".PadLeft(3,'0')+"],");
//         //Console.Write(",");
        
//     }
//     Console.WriteLine();
// }

Console.WriteLine("".PadRight(80,' ')+"PREDICTION");
var predict = Enumerable.Range(1,43).Select(x => $"[{x.ToString().PadLeft(3, '0')}]").ToArray();
Console.WriteLine("    |"+string.Join(",", predict));

Console.WriteLine("".PadRight(171,'-'));

var row = Enumerable.Range(1,43).ToArray();
int i = 0;

foreach (var r in m)
{
    // Convert each 'r' to a string of formatted elements
    var formatted = r.Select(x => $"[{x.ToString().PadLeft(3, '0')}]").ToArray();
    
    Console.Write("|"+ $"{row[i]}".PadLeft(3,'0') +"|");
    i++;
    // Join all formatted elements with a comma and print the result
    Console.WriteLine(string.Join(",", formatted));
}

In [ ]:
// Recall = True Posititive / True Positive + False Negative checj accros X axis
List<float> recall = new ();
for(i = 0;i<m[0].Count();i++)
{
    var s = m[i].Sum();
    

    recall.Add(m[i][i]/(float)s);
}
foreach(var _ in recall ) {
   Console.WriteLine(_);
}


In [ ]:
// Precision = True Positive / True Positive + False Positive  Check accros Y AXIS
List<float> precision = new ();
for(i = 0;i<m[0].Count();i++)
{
    float s = 0;
    for(int j =0 ;j<m[0].Count();j++){
        s+=m[i][j];
    }
    
    precision.Add(m[i][i]/(float)s);
}
foreach(var _ in precision ) {
   Console.WriteLine(_);
}

In [ ]:
//#r "nuget: SandDance.InteractiveExtension,*-*"
//#r "nuget: DataView.InteractiveExtension,*-*"
//#r "nuget: Microsoft.ML.DataView"
//#r "nuget: Microsoft.Data.Analysis"
//#r "nuget: Microsoft.ML, 4.0.0"

In [ ]:
//using Microsoft.Data.Analysis;
//using Microsoft.ML;
//using Microsoft.ML.Data;
//using System.Collections.Generic;